# Vacation PY

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from citipy import citipy
import scipy.stats as sp

# Import API key
from api_key import g_key

# Store Part I results into DataFrame¶¶

Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv("output/data_cities.csv")
df.loc[:, ~df.columns.str.contains('^Unnamed')]

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


# Humidity Heatmap¶
• Configure gmaps. 
• Use the Lat and Lng as locations and Humidity as the weight.
• Add Heatmap layer to map

In [4]:
# Configure Gmaps

gmaps.configure(api_key=g_key)

In [5]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = df[["Lat", "Lng"]].astype(float)
locations

,Lat,Lng
0,68.05,39.51
1,-20.41,57.70
2,77.48,-69.36
3,40.71,112.04
4,-33.93,18.42
...,...,...
543,33.97,36.66
544,24.01,-104.61
545,49.65,0.71
546,51.07,132.56


In [6]:

fig = gmaps.figure()

gmap_layer = gmaps.heatmap_layer(
    locations, 
    weights=df["Humidity"],
    dissipating=False,
    max_intensity=100,
    point_radius = 1
)
    
fig.add_layer(gmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria¶
• Narrow down the cities to fit weather conditions.  
• Drop any rows will null values.

In [7]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
#     - A max temperature lower than 80 degrees but higher than 70.
#     - Wind speed less than 10 mph.
#     - Zero cloudiness.
#     - Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by 
#       your API requests to a reasonable number.

In [8]:
vacation_spot = df[
    (df['Max Temp'] > 70) & 
    (df['Max Temp'] < 80) &
    (df['Wind Speed'] < 10.00) &
    (df['Cloudiness'] > 0) &
    (df['Cloudiness'] < 5)
]
vacation_spot

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07


# Hotel Map
• Store into variable named hotel_df. 
• Add a "Hotel Name" column to the DataFrame. 
• Set parameters to search for hotels with 5000 meters. 
• Hit the Google Places API for each city's coordinates. 
• Store the first Hotel result into the DataFrame. 
• Plot markers on top of the heatmap.

In [9]:
hotel_df = vacation_spot.copy()
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07


In [10]:
# Add a "Hotel Name" column into the DataFrame
hotel_df.loc[:,'Hotel Name'] = np.nan

In [11]:
# Set parameters to search for hotels within 5000 meters
target_radius = 5000
target_type = "hotel"
target_search = "hotel"

In [12]:
# Hit the Google Places API for each city's coordinates (Version 1)

# Add a "Hotel Name" column into the DataFrame
hotel_df.loc[:,'Hotel Name'] = np.nan
# Set parameters to search for hotels within 5000 meters
target_radius = 50000
target_type = "hotel"
target_search = "hotel"

# Hit the Google Places API for each city's coordinates

hotel_list = []

for index, item in hotel_df.iterrows():
    params = {
        "location": f"{item['Lat']}, {item['Lng']}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    
    response = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params).json()
    
    try:
        hotel_list.append(response["results"][0]["name"])
        
    except:
        hotel_list.append("N/A")
        print(f"No hotels within 5km of {coords}.")

hotel_df["Hotel Name"] = hotel_list

In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52,Hotel National
268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73,Sukro
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07,Big 8 Corporate Hotel


In [18]:
# Add marker layer ontop of heat map

new_fig = fig = gmaps.figure()

markers = gmaps.marker_layer(locations,info_box_content= hotel_info)
new_fig.add_layer(gmap_layer)
new_fig.add_layer(markers)
new_fig

Figure(layout=FigureLayout(height='420px'))